In [1]:
import numpy as np
import torch
import tensorflow as tf
import jax
import jax.numpy as jnp
from jax import lax

In [25]:
th_x = torch.randn(1, 3, 9, 9)
tf_x = np.float32(th_x.numpy()).transpose(0, 2, 3, 1)

s = (2, 2) # strides
th_conv = torch.nn.Conv2d(3, 1, 3, s, 0, bias=False)
tf_conv = tf.keras.layers.Convolution2D(1, 3, s, 'valid', use_bias=False)
tf_conv(tf_x)
tf_conv.set_weights([th_conv.weight.data.permute(2, 3, 1, 0).detach().numpy()])

th_y = th_conv(th_x).detach().numpy()
tf_y = tf_conv(tf_x).numpy()
tf_y = tf_y.transpose(0, 3, 1, 2)

print(th_y.shape, th_y.ravel()[:10])
print(tf_y.shape, tf_y.ravel()[:10])
print('Error Rate:', np.mean(np.abs(th_y - tf_y) > 1e-6))

(1, 1, 4, 4) [-0.06813483  0.34486434  0.8420276   0.14421238 -1.0623181   0.07540331
 -0.06762397  0.12286419 -0.10806744  0.10735513]
(1, 1, 4, 4) [-0.06813486  0.34486437  0.84202754  0.1442123  -1.0623181   0.07540331
 -0.06762393  0.12286423 -0.10806744  0.10735514]
Error Rate: 0.0


In [19]:
tf_x.shape

(1, 9, 9, 3)

In [24]:
jax_x = jnp.asarray(th_x.numpy())
jax_k = jnp.asarray(th_conv.weight.data.detach().numpy())

jax_y = lax.conv(
                 jax_x,    # lhs = NCHW image tensor
                 jax_k, # rhs = OIHW conv kernel tensor
                 (s, s),  # window strides
                 'VALID') # padding mode
jax_y = jax_y._value

print(th_y.shape, th_y.ravel()[:10])
print(jax_y.shape, jax_y.ravel()[:10])
print('Error Rate:', np.mean(np.abs(th_y - jax_y) > 1e-6))

TypeError: Shapes must be 1D sequences of concrete values of integer type, got (1, 1, array([7, 7]), array([7, 7])).